This notebook is used to parsing the text data into dataframe.

# Import dependencies

In [4]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import os
import re

# Check the total number of available reviews on sitemap

In [56]:
# Step 1: Get all review URLs from the sitemap
sitemap_url = "https://www.coffeereview.com/sitemap_index.xml"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

# Fetch and parse the sitemap XML
response = requests.get(sitemap_url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, "xml")
    all_urls = [loc.text for loc in soup.find_all("loc")]
    print(f"✅ Found {len(all_urls)} URLs.")
else:
    print(f"❌ Failed to fetch sitemap. Status code: {response.status_code}")
    exit()

✅ Found 15 URLs.


In [57]:
review_sitemaps = [url for url in all_urls if "review-sitemap" in url]
review_sitemaps

['https://www.coffeereview.com/review-sitemap.xml',
 'https://www.coffeereview.com/review-sitemap2.xml',
 'https://www.coffeereview.com/review-sitemap3.xml',
 'https://www.coffeereview.com/review-sitemap4.xml',
 'https://www.coffeereview.com/review-sitemap5.xml',
 'https://www.coffeereview.com/review-sitemap6.xml',
 'https://www.coffeereview.com/review-sitemap7.xml',
 'https://www.coffeereview.com/review-sitemap8.xml',
 'https://www.coffeereview.com/review-sitemap9.xml']

In [58]:
all_review_urls = []

for this_sitemap in review_sitemaps:
    # Fetch and parse the sitemap XML
    response = requests.get(this_sitemap, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "xml")
        review_urls = [loc.text for loc in soup.find_all("loc")]
        print(f"✅ Found {len(review_urls)} review URLs in sitemap {this_sitemap}.")
        all_review_urls = all_review_urls + review_urls
    else:
        print(f"❌ Failed to fetch sitemap. Status code: {response.status_code}")
        exit()

✅ Found 1001 review URLs in sitemap https://www.coffeereview.com/review-sitemap.xml.
✅ Found 1000 review URLs in sitemap https://www.coffeereview.com/review-sitemap2.xml.
✅ Found 1040 review URLs in sitemap https://www.coffeereview.com/review-sitemap3.xml.
✅ Found 1057 review URLs in sitemap https://www.coffeereview.com/review-sitemap4.xml.
✅ Found 1056 review URLs in sitemap https://www.coffeereview.com/review-sitemap5.xml.
✅ Found 1067 review URLs in sitemap https://www.coffeereview.com/review-sitemap6.xml.
✅ Found 1058 review URLs in sitemap https://www.coffeereview.com/review-sitemap7.xml.
✅ Found 1045 review URLs in sitemap https://www.coffeereview.com/review-sitemap8.xml.
✅ Found 429 review URLs in sitemap https://www.coffeereview.com/review-sitemap9.xml.


In [59]:
if "https://www.coffeereview.com/review/" in all_review_urls:
    all_review_urls.remove("https://www.coffeereview.com/review/")

print(f"Found {len(all_review_urls)} review URLs in total.")
print(f"Found {len(set(all_review_urls))} unique review URLs in total.")

Found 8752 review URLs in total.
Found 8751 unique review URLs in total.


## Found duplicated review urls

In [60]:
def first_duplicate(lst):
    seen = set()
    for item in lst:
        if item in seen:
            return item
        seen.add(item)
    return None

print(first_duplicate(all_review_urls))

https://www.coffeereview.com/wp-content/uploads/2014/11/29_375x375.jpg


## Check the scrapped reviews

In [61]:
reviews_from_sitemap = [re.sub(r"[^\w\-]", "_", url) + ".txt" for url in all_review_urls]
reviews_from_sitemap[:5]

['https___www_coffeereview_com_review_100-colombian_.txt',
 'https___www_coffeereview_com_review_moka-java_.txt',
 'https___www_coffeereview_com_review_java_.txt',
 'https___www_coffeereview_com_review_sumatra-gayo-mountain_.txt',
 'https___www_coffeereview_com_review_folgers-french-roast_.txt']

In [62]:
folder_path = "./coffee_reviews_text/"
scrapped_texts = os.listdir(folder_path)
print(len(scrapped_texts))

8747


In [63]:
# Check differences between reviews_from_sitemap and scrapped_texts
set_1 = set(reviews_from_sitemap)
set_2 = set(scrapped_texts)

set_1 ^ set_2

{'https___www_coffeereview_com_review_100-arabica-nespresso-compatible-capsules_.txt',
 'https___www_coffeereview_com_review_colombia-manos-juntas-anaerobic-natural_.txt',
 'https___www_coffeereview_com_review_colombia-sebastian-ramirez-washed-pink-bourbon-2_.txt',
 'https___www_coffeereview_com_wp-content_uploads_2019_02_6_375x375_jpg.txt'}

In [64]:
# Remove 360 wp-content files
scrapped_reviews = [filename for filename in scrapped_texts if "wp-content" not in filename]
print(f"We will use {len(scrapped_reviews)} unique reviews for analysis.")

We will use 8387 unique reviews for analysis.


This number should align with the number of reviews (until 03/03/2025) on the website https://www.coffeereview.com/review/:
$20 \times 414 + 4 = 8284$. However, it looks like we scrapped more reviews ($8387 > 8284$) than those shown on the website.

## No need to run again - Read all text files and save as csv with raw texts which need to be parsed further
Done by Xin on 03/03/2025

In [25]:
# Define regex patterns
url_pattern = re.compile(r'URL:\s*(https?://\S+)')
all_text_pattern = re.compile(r'“行銷攻略” 促銷活動\s*(.*?)\s*Explore Similar Coffees', re.DOTALL)

def extract_info(text):
    """Extract URL and relevant text from the review file"""
    url = url_pattern.search(text)
    all_text = all_text_pattern.search(text)
    
    return {
        "URL": url.group(1) if url else None,
        "all_text": all_text.group(1).strip() if all_text else None,
    }

data = []
for file_name in scrapped_reviews:
    with open(os.path.join(folder_path, file_name), "r", encoding="utf-8") as file:
        text = file.read()
        extracted_info = extract_info(text)
        data.append(extracted_info)

# Store data in DataFrame
df = pd.DataFrame(data)

In [ ]:
# check_index = 8386
# print(df["URL"][check_index])
# print(df["all_text"][check_index])

https://www.coffeereview.com/review/__trashed-5/
94
JBC Coffee Roasters
Kagunyu Kenya
Roaster Location:
Madison, Wisconsin
Coffee Origin:
Nyeri County, Kenya
Roast Level:
Medium-Light
Agtron:
60/78
Est. Price:
$22.00/12 ounces
Review Date:
February 2024
Aroma:
9
Acidity/Structure:
9
Body:
9
Flavor:
9
Aftertaste:
8
Blind Assessment
Complex, multi-layered, deep-toned. Red currant, cocoa nib, tangerine, fresh-cut oak, marjoram in aroma and cup. Bright, juicy structure with phosphoric (cola-like) acidity; crisp, syrupy mouthfeel. Resonant finish centered around notes of red currant and cocoa nib.
Notes
Produced by smallholding farmers, from trees of the SL28 and SL34 varieties of Arabica, and processed by the traditional washed method (fruit skin and pulp removed before drying) at the Kagunyu Washing Station. JBC Coffee Roasters’ vision is simple: “Let the coffee lead the way” through sourcing and roasting the best and most unique coffees available and rewarding the farmers who grow those 

In [33]:
df.to_csv("coffee_review_raw_texts.csv", index = False)

# Start here! Raw Text Parsing

In [5]:
data = pd.read_csv("coffee_review_raw_texts.csv")
data

,URL,all_text
0,https://www.coffeereview.com/review/100-arabic...,89\nCaffe Bomrad\n100% Arabica 100% Italiano\n...
1,https://www.coffeereview.com/review/100-arabic...,"87\nLucaff?\n100% Arabica, Black Label (ESE po..."
2,https://www.coffeereview.com/review/100-arabic...,87\nCaribeans\n100% Arabica Coffee from Puerto...
3,https://www.coffeereview.com/review/100-arabic...,88\nWaka Coffee\n100% Arabica Freeze-Dried Col...
4,https://www.coffeereview.com/review/100-arabic...,72\nYuban\n100% Arabica Instant Coffee\nRoaste...
...,...,...
8382,https://www.coffeereview.com/review/zimbabwean...,88\nLeopard Forest Coffee\nZimbabwean Peaberry...
8383,https://www.coffeereview.com/review/zimbabwe/,83\nThe Sensuous Bean\nZimbabwe\nRoaster Locat...
8384,https://www.coffeereview.com/review/zombie-des...,87\nCafe Kreyol\nZombie Desert 100% Organic Ha...
8385,https://www.coffeereview.com/review/zoom-espre...,93\nZuco Coffee Roasters\nZoom Espresso\nRoast...


In [37]:
# Observation on 1 text
# 76 - Bottom Line; 3333 - Who Should Drink It
check_index = 76
print(data["all_text"][check_index])
# data["all_text"][check_index]

90
Coffee by Design
1994 Blend
Roaster Location:
Portland, Maine
Coffee Origin:
Ethiopia
Roast Level:
Medium-Light
Agtron:
57/74
Est. Price:
$20.50/16 ounces
Review Date:
September 2023
Aroma:
8
Acidity/Structure:
8
Body:
8
Flavor:
8
Aftertaste:
8
Blind Assessment
Richly fruit-toned, wood-framed. Mulberry, cedar, Meyer lemon zest, cane sugar, lily in aroma and cup. Pert acidity; crisp, satiny mouthfeel. Finish consolidates to cedar and lily.
Notes
A blend of coffees from Ethiopia, both washed and natural-processed. Founded in 1994, Coffee By Design is a specialty coffee roaster in Portland, Maine that also operates three coffee shops in Portland. For more information, visit
www.coffeebydesign.com.
Bottom Line
Pleasing aromatic cedar notes frame this fruit-driven blend of coffees from Ethiopia: berry-forward and gently spice-toned.


## Define extraction patterns

Pattern concerns:

1. Some reviews do not have "Bottom Line" or "Who Should Drink It"
2. Some do not have "Aftertaste"



In [69]:
def check_variable(variable):
    print(f"There are {sum(~data['all_text'].str.contains(variable))} reviews which do not contain {variable}")

variables = ["Roaster Location", "Coffee Origin", "Roast Level", "Agtron", 
             "Est. Price", "Review Date", "Aroma", "Acidity", "Body", 
             "Flavor", "Aftertaste", "With Milk",
             "Blind Assessment", "Notes", "Who Should Drink It", "Bottom Line"]

for variable in variables:
    check_variable(variable)

There are 104 reviews which do not contain Roaster Location
There are 572 reviews which do not contain Coffee Origin
There are 103 reviews which do not contain Roast Level
There are 103 reviews which do not contain Agtron
There are 2015 reviews which do not contain Est. Price
There are 0 reviews which do not contain Review Date
There are 164 reviews which do not contain Aroma
There are 1375 reviews which do not contain Acidity
There are 114 reviews which do not contain Body
There are 115 reviews which do not contain Flavor
There are 973 reviews which do not contain Aftertaste
There are 7242 reviews which do not contain With Milk
There are 0 reviews which do not contain Blind Assessment
There are 0 reviews which do not contain Notes
There are 4360 reviews which do not contain Who Should Drink It
There are 4181 reviews which do not contain Bottom Line


## Start parsing

In [104]:
# Define regex patterns
rating_pattern = re.compile(r'^(\d{2,3})\n')
roaster_pattern = re.compile(r'^(\d{2,3})\n(.*?)\n', re.MULTILINE)
coffee_name_pattern = re.compile(r'^(\d{2,3})\n.*?\n(.*?)\n', re.MULTILINE)
location_pattern = re.compile(r'Roaster Location:\s*(.*?)\n')
origin_pattern = re.compile(r'Coffee Origin:\s*(.*?)\n')
roast_pattern = re.compile(r'Roast Level:\s*(.*?)\n')
agtron_pattern = re.compile(r'Agtron:\s*(.*?)\n')
price_pattern = re.compile(r'Est. Price:\s*(.*?)\n')
date_pattern = re.compile(r'Review Date:\s*(.*?)\n')
score_pattern = re.compile(r'(Aroma|Acidity|Acidity/Structure|Body|Flavor|Aftertaste|With Milk|Flavor in milk):\s*(\d+)')
section_pattern = re.compile(
    r"(Blind Assessment|Notes|Who Should Drink It|Bottom Line)\n(.*?)(?=(?:Blind Assessment|Notes|Who Should Drink It|Bottom Line|$))",
    re.DOTALL
)

def extract_info(text):
    """Extract structured data from the review text"""
    rating = rating_pattern.search(text)
    roaster = roaster_pattern.search(text)
    coffee_name = coffee_name_pattern.search(text)
    location = location_pattern.search(text)
    origin = origin_pattern.search(text)
    roast = roast_pattern.search(text)
    agtron = agtron_pattern.search(text)
    price = price_pattern.search(text)
    date = date_pattern.search(text)
    scores = {match[0]: int(match[1]) for match in score_pattern.findall(text)}

     # Extract sections dynamically
    section_matches = section_pattern.findall(text)
    section_data = {match[0]: match[1].strip() for match in section_matches}

    
    return {
        "Rating": int(rating.group(1)) if rating else None,
        "Roaster": roaster.group(2).strip() if roaster else None,
        "Coffee Name": coffee_name.group(2).strip() if coffee_name else None,
        "Roaster Location": location.group(1) if location else None,
        "Coffee Origin": origin.group(1) if origin else None,
        "Roast Level": roast.group(1) if roast else None,
        "Agtron": agtron.group(1) if agtron else None,
        "Est. Price": price.group(1) if price else None,
        "Review Date": date.group(1) if date else None,
        "Aroma": scores.get("Aroma"),
        "Acidity": scores.get("Acidity"),
        "Acidity/Structure": scores.get("Acidity/Structure"),
        "Body": scores.get("Body"),
        "Flavor": scores.get("Flavor"),
        "Aftertaste": scores.get("Aftertaste"),
        "With Milk": scores.get("With Milk"),
        "Flavor in milk": scores.get("Flavor in milk"),
        "Blind Assessment": section_data.get("Blind Assessment"),
        "Notes": section_data.get("Notes"),
        "Who Should Drink It": section_data.get("Who Should Drink It"),
        "Bottom Line": section_data.get("Bottom Line"),
    }

## Try the parsing function on 1 text

In [105]:
# Try to parse 1 text
check_index = 76
text = data["all_text"][check_index]
extract_info(text)

{'Rating': 90,
 'Roaster': 'Coffee by Design',
 'Coffee Name': '1994 Blend',
 'Roaster Location': 'Portland, Maine',
 'Coffee Origin': 'Ethiopia',
 'Roast Level': 'Medium-Light',
 'Agtron': '57/74',
 'Est. Price': '$20.50/16 ounces',
 'Review Date': 'September 2023',
 'Aroma': 8,
 'Acidity': None,
 'Acidity/Structure': 8,
 'Body': 8,
 'Flavor': 8,
 'Aftertaste': 8,
 'With Milk': None,
 'Flavor in milk': None,
 'Blind Assessment': 'Richly fruit-toned, wood-framed. Mulberry, cedar, Meyer lemon zest, cane sugar, lily in aroma and cup. Pert acidity; crisp, satiny mouthfeel. Finish consolidates to cedar and lily.',
 'Notes': 'A blend of coffees from Ethiopia, both washed and natural-processed. Founded in 1994, Coffee By Design is a specialty coffee roaster in Portland, Maine that also operates three coffee shops in Portland. For more information, visit\nwww.coffeebydesign.com.',
 'Who Should Drink It': None,
 'Bottom Line': 'Pleasing aromatic cedar notes frame this fruit-driven blend of cof

## Parsing all text

In [141]:
# Process all text from DataFrame
data["parsed_data"] = data["all_text"].apply(extract_info)

# Expand dictionary columns into separate DataFrame columns
parsed_df = pd.DataFrame(data["parsed_data"].tolist())

# Combine parsed data with original DataFrame (keeping URL as identifier)
result_df = pd.concat([data[["URL", "all_text"]], parsed_df], axis=1)

In [142]:
result_df.columns

Index(['URL', 'all_text', 'Rating', 'Roaster', 'Coffee Name',
       'Roaster Location', 'Coffee Origin', 'Roast Level', 'Agtron',
       'Est. Price', 'Review Date', 'Aroma', 'Acidity', 'Acidity/Structure',
       'Body', 'Flavor', 'Aftertaste', 'With Milk', 'Flavor in milk',
       'Blind Assessment', 'Notes', 'Who Should Drink It', 'Bottom Line'],
      dtype='object')

# Check missing values
This result should align with the string search result before parsing.

In [143]:
for variable in variables:
    check_variable(variable)

There are 104 reviews which do not contain Roaster Location
There are 572 reviews which do not contain Coffee Origin
There are 103 reviews which do not contain Roast Level
There are 103 reviews which do not contain Agtron
There are 2015 reviews which do not contain Est. Price
There are 0 reviews which do not contain Review Date
There are 164 reviews which do not contain Aroma
There are 1375 reviews which do not contain Acidity
There are 114 reviews which do not contain Body
There are 115 reviews which do not contain Flavor
There are 973 reviews which do not contain Aftertaste
There are 7242 reviews which do not contain With Milk
There are 0 reviews which do not contain Blind Assessment
There are 0 reviews which do not contain Notes
There are 4360 reviews which do not contain Who Should Drink It
There are 4181 reviews which do not contain Bottom Line


In [144]:
result_df.isnull().sum()

URL                       0
all_text                  0
Rating                    5
Roaster                   5
Coffee Name               5
Roaster Location        104
Coffee Origin           572
Roast Level             103
Agtron                  103
Est. Price             2015
Review Date               0
Aroma                   167
Acidity                4679
Acidity/Structure      5117
Body                    116
Flavor                  120
Aftertaste              974
With Milk              7250
Flavor in milk         8379
Blind Assessment          0
Notes                     0
Who Should Drink It    4360
Bottom Line            4181
dtype: int64

Some numbers of missing values does not align with those in the raw text. Check why:

Aroma, Body, Flavor, Aftertaste, With Milk, 
(Acidity, Acidity/Structure)

In [145]:
def get_missing_urls(which_variable, df = result_df):
    """This function is used to print out all the urls 
    where 'which_variable' exists in the raw text but have no values in the corresponding column."""
    parse_missing = df[df[which_variable].isnull()]
    raw_not_missing = parse_missing[parse_missing["all_text"].str.contains(which_variable)]["URL"]
    for url in raw_not_missing:
        print(url)

## Check Aroma
Suppose to miss 164 scores but missed 167 scores. After checking, 3 reviews are actually missing Aroma scores due to some reason. So parsing for Aroma is correct!

In [111]:
get_missing_urls("Aroma")

https://www.coffeereview.com/review/el-salvador-2/
https://www.coffeereview.com/review/natural-moka-green/
https://www.coffeereview.com/review/wilton-benitez-colombia-variety-p-01/


## Check Body
Suppose to miss 114 scores but missed 116 scores. After checking 2 reviews are actually missing Body scores due to some reason. So parsing for Body is correct!

In [113]:
get_missing_urls("Body")

https://www.coffeereview.com/review/el-salvador-2/
https://www.coffeereview.com/review/natural-moka-green/


## Check Flavor
Suppose to miss 115 scores but missed 120 scores. After checking 5 reviews are actually missing Flavor scores due to some reason. So parsing for Flavor is correct!

In [114]:
get_missing_urls("Flavor")

https://www.coffeereview.com/review/el-salvador-2/
https://www.coffeereview.com/review/natural-moka-green/
https://www.coffeereview.com/review/twisted/
https://www.coffeereview.com/review/wilton-benitez-colombia-geisha-p-06/
https://www.coffeereview.com/review/wilton-benitez-colombia-orange-bourbon-p-17/


## Check Aftertaste
Suppose to miss 973 scores but missed 974 scores. After checking, 1 review is actually missing Afterstate score due to some reason. So parsing for Aftertaste is correct!

In [115]:
get_missing_urls("Aftertaste")

https://www.coffeereview.com/review/papua-new-guinea-purosa/


## Check With Milk
Suppose to miss 7242 scores but missed 7250 scores. After checking, 8 review actually have With Milk scores but the pattern is different whick looks like:

With Milk:
Flavor in milk: 5

Therefore, I change add 1 parsing pattern for "Flavor in milk".

In [116]:
get_missing_urls("With Milk")

https://www.coffeereview.com/review/espresso-blend/
https://www.coffeereview.com/review/espresso-dolce/
https://www.coffeereview.com/review/illy-espresso/
https://www.coffeereview.com/review/major-dickasons-blend-2/
https://www.coffeereview.com/review/napoli-blend/
https://www.coffeereview.com/review/perugia-blend-2/
https://www.coffeereview.com/review/perugia-blend/
https://www.coffeereview.com/review/qualita-oro/


In [117]:
print(result_df[result_df["URL"] == "https://www.coffeereview.com/review/espresso-blend/"]["all_text"][2417])

77
Bucks County Coffee
Espresso Blend
Roaster Location:
Langhorne, Pennsylvania
Roast Level:
Dark
Agtron:
36/38
Review Date:
October 1997
Aroma:
6
Body:
5
Flavor:
4
With Milk:
Flavor in milk: 5
Blind Assessment
A carbony bite suggests this blend may have been roasted too quickly. Otherwise a very agreeable, rather straightforward Espresso, with solid body and a satisfying balance of sweet and pungent tones. The profile fades rather quickly in the finish, perhaps owing to a rushed roast that burned off aromatics.
Notes
A very agreeable, rather straightforward espresso, with solid body and a satisfying balance of sweet and pungent tones. Bucks County Coffee is a specialty roaster and retail coffee chain based in Pennsylvania.
Who Should Drink It
Those who sugar their cup or who like a subdued twist of carbon in an otherwise classic demitasse.


## Check Acidity
Suppose to miss 1375 scores but missed 1409 scores in total after combining 'Acidity' and 'Acidity/Structure' together.

Some coffee reviews contains Acidity scores, but others contains Acidity/Structure scores.

25 reviews are actually missing 'Acidity' scores OR 'Acidity/Structure' scores.

9 reviews describe Acidity in words:
| URL | Acidity |
|:----------|:----------:|
| https://www.coffeereview.com/review/aged-sumatra-2/ | Very Low |
| https://www.coffeereview.com/review/blue-de-brasil/ | Low |
| https://www.coffeereview.com/review/classique-high-roasted/ | Low |
| https://www.coffeereview.com/review/guatemala-antigua-with-coffee-tamer-added/ | Moderate |
| https://www.coffeereview.com/review/india-monsoon-malabar/ | Very Low |
| https://www.coffeereview.com/review/indian-mysore-plantation/ | Low |
| https://www.coffeereview.com/review/low-acid-arabica/ | Moderate |
| https://www.coffeereview.com/review/sumatra-6/ | Very Low |
| https://www.coffeereview.com/review/tamer-low-acid-coffee-beans-regular/ | Very Low |



In [139]:
# Check if there're conflicts like have values in both Flavor in milk and With Milk columns
# No conflicts!
result_df[(~result_df["Acidity"].isnull()) & (~result_df["Acidity/Structure"].isnull())] # Expected: 0 rows

,URL,all_text,Rating,Roaster,Coffee Name,Roaster Location,Coffee Origin,Roast Level,Agtron,Est. Price,...,Acidity,Acidity/Structure,Body,Flavor,Aftertaste,With Milk,Blind Assessment,Notes,Who Should Drink It,Bottom Line


In [126]:
result_df["Combined_Acidity"] = result_df["Acidity"].fillna(result_df["Acidity/Structure"])

In [132]:
parse_missing = result_df[result_df["Combined_Acidity"].isnull()]
raw_not_missing = parse_missing[parse_missing["all_text"].str.contains("Acidity")]["URL"]
print(len(raw_not_missing))

# for i, url in enumerate(raw_not_missing):
#     print(i+1, end=" ")
#     print(url)

34


# Save the correctly parsed dataframe

## Handle before save: Merge Flavor in milk into With milk

In [146]:
# Check if there're conflicts like have values in both Flavor in milk and With Milk columns
# No conflicts!
result_df[(~result_df["Flavor in milk"].isnull()) & (~result_df["With Milk"].isnull())] # Expected: 0 rows

,URL,all_text,Rating,Roaster,Coffee Name,Roaster Location,Coffee Origin,Roast Level,Agtron,Est. Price,...,Acidity/Structure,Body,Flavor,Aftertaste,With Milk,Flavor in milk,Blind Assessment,Notes,Who Should Drink It,Bottom Line


In [147]:
# Merge Flavor in milk into With milk
result_df["With Milk"] = result_df["With Milk"].fillna(result_df["Flavor in milk"])
assert sum(result_df["With Milk"].isnull()) == 7242

result_df = result_df.drop(columns=['Flavor in milk'])

In [149]:
result_df.describe()

,Rating,Aroma,Acidity,Acidity/Structure,Body,Flavor,Aftertaste,With Milk
count,8382.000000,8220.000000,3708.000000,3270.000000,8271.000000,8267.000000,7413.000000,1145.000000
mean,91.121928,8.425304,7.629450,8.504587,8.172410,8.520503,7.997842,8.491703
std,4.070747,0.883198,1.143691,0.617238,0.845941,0.959832,0.701548,0.772207
min,52.000000,2.000000,1.000000,1.000000,4.000000,1.000000,2.000000,5.000000
25%,90.000000,8.000000,7.000000,8.000000,8.000000,8.000000,8.000000,8.000000
50%,92.000000,9.000000,8.000000,9.000000,8.000000,9.000000,8.000000,9.000000
75%,94.000000,9.000000,8.000000,9.000000,9.000000,9.000000,8.000000,9.000000
max,98.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


In [150]:
result_df.isnull().sum()

URL                       0
all_text                  0
Rating                    5
Roaster                   5
Coffee Name               5
Roaster Location        104
Coffee Origin           572
Roast Level             103
Agtron                  103
Est. Price             2015
Review Date               0
Aroma                   167
Acidity                4679
Acidity/Structure      5117
Body                    116
Flavor                  120
Aftertaste              974
With Milk              7242
Blind Assessment          0
Notes                     0
Who Should Drink It    4360
Bottom Line            4181
dtype: int64

In [148]:
result_df.to_csv("coffee_reviews_parsed.csv", index=False)